# Clustering of Wholesale Customers

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
wholesale_customer_data_csv = pd.read_csv('Wholesale_customers_data.csv',sep=',')
dataset = wholesale_customer_data_csv
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Channel           440 non-null    int64
 1   Region            440 non-null    int64
 2   Fresh             440 non-null    int64
 3   Milk              440 non-null    int64
 4   Grocery           440 non-null    int64
 5   Frozen            440 non-null    int64
 6   Detergents_Paper  440 non-null    int64
 7   Delicassen        440 non-null    int64
dtypes: int64(8)
memory usage: 27.6 KB
